# import

In [1]:
import torch
from bart_model_from_scratch.multihead_attn import BartAttention
from transformers import BartConfig

/home/phamngocthi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# BartConfig

In [2]:
config = BartConfig()
config.pad_token_id = 2
config.encoder_layerdrop = 0.1
config.decoder_layerdrop = 0.1
config.d_model = config.encoder_attention_heads

# BartAttention

In [3]:
bart_attn = BartAttention(
    embed_dim=config.d_model,
    num_heads=config.encoder_attention_heads,
    dropout=config.attention_dropout,
)
print(bart_attn)

BartAttention(
  (k_proj): Linear(in_features=16, out_features=16, bias=True)
  (v_proj): Linear(in_features=16, out_features=16, bias=True)
  (q_proj): Linear(in_features=16, out_features=16, bias=True)
  (out_proj): Linear(in_features=16, out_features=16, bias=True)
)


In [4]:
# test bart_attn
hidden_states = torch.randn(2, 4, config.d_model)
output = bart_attn(hidden_states)
print(output.shape)
print(output)

torch.Size([2, 4, 16])
tensor([[[ 1.0462,  0.3592,  0.2246, -0.5626,  0.1817,  0.5669,  0.6548,
           0.5279, -0.2823, -0.3640,  1.2936, -1.0978,  0.2974, -0.6205,
          -1.0247, -0.0346],
         [-0.3718, -0.1760,  0.3206,  0.0215,  0.2763, -0.4573,  0.0340,
          -0.7747,  0.6343, -0.1275,  0.3659,  0.6905, -0.4680,  0.4454,
           0.1758, -0.4128],
         [-0.3091,  0.1989,  0.6613,  0.4054, -0.0936, -0.2836, -0.5086,
          -0.6263,  0.9526, -0.1183, -0.0350,  0.6673, -0.5660,  0.4035,
           0.4103, -0.0595],
         [-0.5635,  0.0474,  0.2046,  0.2334,  0.5169, -0.8230,  0.2165,
          -0.5140,  0.4842, -0.0744,  0.0973,  0.8233, -0.4679,  0.7096,
           0.4349, -0.2063]],

        [[-0.2984,  0.3075, -0.0122,  0.1216, -0.0853,  0.2248,  0.1031,
          -0.1167, -0.2619,  0.1801, -0.0073,  0.1607, -0.2764,  0.2743,
           0.2203,  0.0849],
         [-0.0372,  0.3480, -0.1320, -0.0014,  0.1742,  0.2192,  0.4463,
           0.3129, -0.2551,

# BartEncoderLayer

In [5]:
from bart_model_from_scratch.encoder_layer import BartEncoderLayer

In [6]:
bart_encoder_layer = BartEncoderLayer(config)
bart_encoder_layer

BartEncoderLayer(
  (self_attn): BartAttention(
    (k_proj): Linear(in_features=16, out_features=16, bias=True)
    (v_proj): Linear(in_features=16, out_features=16, bias=True)
    (q_proj): Linear(in_features=16, out_features=16, bias=True)
    (out_proj): Linear(in_features=16, out_features=16, bias=True)
  )
  (self_attn_layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (activation_fn): GELU(approximate='none')
  (activation_dropout): Dropout(p=0.0, inplace=False)
  (fc1): Linear(in_features=16, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=16, bias=True)
  (final_layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
)

In [7]:
# test bart_encoder_layer
hidden_states = torch.randn(2, 4, config.d_model, dtype=torch.float32)
print(hidden_states.shape)
output = bart_encoder_layer(hidden_states)
print(output.shape)
print(output)

torch.Size([2, 4, 16])
torch.Size([2, 4, 16])
tensor([[[ 1.8030e+00, -2.3779e-01,  7.2607e-01, -2.1844e+00, -1.8298e-01,
           2.4605e-01, -1.0806e+00,  1.0848e+00, -2.6457e-01, -1.2902e+00,
           8.7900e-02, -1.0991e-01,  2.8431e-01, -3.1176e-01,  1.7152e+00,
          -2.8511e-01],
         [-1.2394e+00, -1.7937e+00,  1.6529e+00,  8.2487e-02, -4.2537e-01,
           5.5730e-01, -1.0607e+00, -2.3867e-02,  7.4978e-01,  1.2225e+00,
           4.7909e-01, -5.3035e-01,  1.5226e+00,  2.1020e-01, -1.4000e+00,
          -3.4838e-03],
         [-1.9641e-01, -1.0728e+00, -9.2217e-01,  1.0100e+00, -6.7216e-01,
           7.4513e-01,  2.7732e-01, -1.2440e+00, -1.0530e+00,  1.7775e+00,
           1.8320e+00, -1.4973e+00,  6.0952e-01,  7.7392e-02,  2.3515e-01,
           9.3876e-02],
         [-1.1667e+00,  5.8067e-01, -7.1737e-01,  2.8107e-01,  8.7349e-01,
           1.1116e-01,  2.6952e-01,  9.9211e-01, -1.1643e-01,  9.3360e-01,
           1.3948e+00,  2.4243e-02, -9.6769e-01, -8.3995e

# BartDecoderLayer

In [8]:
from bart_model_from_scratch.decoder_layer import BartDecoderLayer

In [9]:
bart_decoder_layer = BartDecoderLayer(config)
bart_decoder_layer

BartDecoderLayer(
  (self_attn): BartAttention(
    (k_proj): Linear(in_features=16, out_features=16, bias=True)
    (v_proj): Linear(in_features=16, out_features=16, bias=True)
    (q_proj): Linear(in_features=16, out_features=16, bias=True)
    (out_proj): Linear(in_features=16, out_features=16, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (activation_fn): GELU(approximate='none')
  (activation_dropout): Dropout(p=0.0, inplace=False)
  (self_attn_layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (encoder_attn): BartAttention(
    (k_proj): Linear(in_features=16, out_features=16, bias=True)
    (v_proj): Linear(in_features=16, out_features=16, bias=True)
    (q_proj): Linear(in_features=16, out_features=16, bias=True)
    (out_proj): Linear(in_features=16, out_features=16, bias=True)
  )
  (encoder_attn_layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=16, out_features=4096, bias=True)
  (fc2): Linear(in_feat

In [10]:
# test bart_decoder_layer
hidden_states = torch.randn(2, 4, config.d_model, dtype=torch.float32)
encoder_hidden_states = torch.randn(2, 4, config.d_model, dtype=torch.float32)
print(hidden_states.shape)
print(encoder_hidden_states.shape)
output = bart_decoder_layer(
    hidden_states=hidden_states,
    encoder_hidden_states=encoder_hidden_states,
)
print(output.shape)
print(output)

torch.Size([2, 4, 16])
torch.Size([2, 4, 16])
torch.Size([2, 4, 16])
tensor([[[-1.5444e+00, -1.8321e+00, -4.1857e-01,  1.5340e+00, -8.1430e-01,
           1.8727e-03,  2.0293e+00, -3.5316e-02,  3.0787e-02,  8.0391e-02,
           1.0235e+00, -7.0789e-01,  3.7463e-01,  2.2874e-01, -7.4908e-01,
           7.9839e-01],
         [ 9.7598e-01,  1.2249e+00, -1.4192e+00,  4.6237e-01,  1.3616e+00,
           2.7617e-01, -1.3110e+00,  3.1597e-01, -2.3389e+00, -1.5550e-01,
          -2.8599e-01,  9.2186e-01, -5.4749e-01, -5.3114e-01,  4.5421e-01,
           5.9622e-01],
         [ 2.5801e-01, -1.3298e+00, -1.9878e-01, -2.0574e-01, -1.1998e+00,
           1.6933e+00,  5.9514e-01,  8.1902e-01,  1.1743e+00, -9.5891e-01,
           2.0939e-01, -4.1938e-01,  4.4432e-01, -2.1860e+00,  2.0231e-01,
           1.1027e+00],
         [ 1.3444e+00, -3.1018e-01, -7.0288e-01,  7.6817e-02,  1.5185e+00,
          -7.6392e-01, -1.6996e+00,  2.5525e-01, -4.3272e-01,  6.4973e-01,
           6.1103e-01,  1.5664e+00

# BartEmbeds

In [11]:
from bart_model_from_scratch.embeds import BartEmbeds

In [12]:
config.src_vocab_size = 50265
config.tgt_vocab_size = 50265
bart_embeds = BartEmbeds(
    num_embeddings=config.src_vocab_size,
    embedding_dim=config.d_model,
    padding_idx=config.pad_token_id,
    max_position_embeddings=config.max_position_embeddings,
)

In [13]:
# test BartEmbeds
input_ids = torch.randint(0, config.src_vocab_size, (2, 4))
output = bart_embeds(input_ids)
print(output.shape)
print(output)

torch.Size([2, 4, 16])
tensor([[[-2.4160e+00, -1.9661e+00,  2.6443e+00, -9.3280e-01,  2.4465e+00,
           2.4268e-01,  1.7166e+00,  2.3802e+00,  1.3417e+00, -5.4644e-01,
          -1.1550e+00, -1.1335e+00,  1.3520e+00,  1.0609e+00,  7.9171e-01,
           9.2121e-01],
         [-4.5567e-01, -4.8341e-01,  8.6834e-03,  3.6270e+00,  1.8820e+00,
          -1.5477e+00, -7.3921e-01,  7.3150e-01, -5.2626e-01,  1.2476e+00,
          -3.4702e-01,  1.3448e+00,  4.2299e-01, -1.9006e+00, -2.0581e-01,
           1.8824e-01],
         [ 3.1356e-01, -4.2852e-01,  3.4789e-01,  1.5724e+00,  6.7705e-01,
           5.0431e-01, -3.0831e-01, -5.6241e-01, -1.8508e-01,  2.2599e+00,
           2.6357e+00,  8.5528e-01,  5.7900e-01, -1.1962e+00,  1.2859e+00,
           1.8313e+00],
         [-9.6787e-01, -1.2522e+00, -5.0352e-01, -6.1392e-01, -4.1005e+00,
           7.1405e-01,  8.3517e-01,  6.4711e-01,  2.5311e+00, -2.9036e+00,
           2.1044e+00,  2.6219e-01, -1.7285e+00, -1.4205e-02, -1.1527e+00,
     

# utils.mask.create_encoder_mask

In [14]:
from bart_model_from_scratch.utils.mask import (
    create_encoder_atn_mask,
)

In [15]:
# test create_encoder_mask
input_ids = torch.randint(0, 10, (5, 4)).to(torch.float32)
input_ids

tensor([[6., 6., 3., 2.],
        [5., 1., 3., 3.],
        [2., 5., 3., 8.],
        [8., 4., 2., 0.],
        [4., 7., 9., 7.]])

In [16]:
attention_mask = (input_ids != config.pad_token_id).to(torch.int64)
attention_mask

tensor([[1, 1, 1, 0],
        [1, 1, 1, 1],
        [0, 1, 1, 1],
        [1, 1, 0, 1],
        [1, 1, 1, 1]])

In [17]:
encoder_attention_mask = create_encoder_atn_mask(
    attention_mask=attention_mask,
    dtype=input_ids.dtype,
)

In [18]:
print(encoder_attention_mask.shape)
print(encoder_attention_mask)

torch.Size([5, 1, 4, 4])
tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]],


        [[[-3.4028e+38,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-3.4028e+38,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-3.4028e+38,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-3.4028e+38,  0.0000e+00,  0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00, -3.4028e+38,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38,  0.0000e+00],
   

# BartEncoder

In [19]:
from bart_model_from_scratch.encoder import BartEncoder

In [20]:
bart_encoder = BartEncoder(config)

In [21]:
# test bart_encoder
input_embeds = torch.randn(2, 4, config.d_model)
# attention_mask = torch.randint(0, 2, (2, 4))
attention_mask = torch.tensor(
    [
        [1, 1, 1, 1],
        [1, 1, 1, 0],
    ]
)
encoder_mask = create_encoder_atn_mask(
    attention_mask=attention_mask,
    dtype=input_embeds.dtype,
)
# print(f"{encoder_mask=}")
# print(f"{input_embeds.shape=}, {attention_mask.shape=}")
# print(f"{input_embeds=}, {attention_mask=}")
output = bart_encoder(input_embeds, attention_mask)
# print(output.shape)
print(output)

tensor([[[ 1.5404, -1.9294, -1.0220, -0.7080, -1.1214,  1.3517, -0.0978,
           0.0620, -0.4251,  1.3274,  0.2395,  0.3164, -0.1805,  1.0341,
          -1.1895,  0.8022],
         [ 0.6196, -1.1127, -0.8184,  0.9903, -0.4060,  1.0541, -1.4214,
          -0.8607, -0.8077, -0.2565,  1.2298, -0.6836, -0.3138,  1.1268,
          -0.4438,  2.1040],
         [ 0.5989, -0.5086, -0.5109,  0.9121, -0.6072, -0.0942, -0.6524,
           0.8949, -0.2833,  0.5651,  1.8712, -0.1361, -0.2460,  0.7344,
          -2.8415,  0.3035],
         [ 0.8179, -0.6423, -0.4396,  0.6302,  0.7321, -0.3719, -0.9057,
           0.1630, -0.5038, -0.7877,  1.9304,  0.5566,  0.1752,  1.0278,
          -2.6003,  0.2183]],

        [[-0.9234,  0.6368, -0.7670,  0.1170, -2.0568,  0.8087, -0.3666,
           0.8740, -1.3334,  0.3223, -0.1272, -0.3249,  1.2975,  1.1477,
          -0.8994,  1.5949],
         [-0.9555,  0.7023, -0.8008,  0.1991, -2.1627,  1.2012, -0.3068,
           0.8193, -0.3725, -0.9814,  0.0968, -0.3

# utils.mask.create_decoder_mask

In [22]:
from bart_model_from_scratch.utils.mask import (
    create_decoder_atn_mask
)

In [23]:
# test causal_mask
attention_mask = torch.tensor([
    [1, 1, 1, 0, 0],
    [1, 1, 0, 0, 0]
])
dtype = torch.float32
create_decoder_atn_mask(
    attention_mask=attention_mask,
    dtype=dtype,
)

tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38]]],


        [[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38]]]])

# BartDecoder

In [24]:
from bart_model_from_scratch.decoder import BartDecoder

In [25]:
bart_decoder = BartDecoder(config)
bart_decoder

BartDecoder(
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-11): 12 x BartDecoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=16, out_features=16, bias=True)
        (v_proj): Linear(in_features=16, out_features=16, bias=True)
        (q_proj): Linear(in_features=16, out_features=16, bias=True)
        (out_proj): Linear(in_features=16, out_features=16, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (activation_fn): GELU(approximate='none')
      (activation_dropout): Dropout(p=0.0, inplace=False)
      (self_attn_layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): BartAttention(
        (k_proj): Linear(in_features=16, out_features=16, bias=True)
        (v_proj): Linear(in_features=16, out_features=16, bias=True)
        (q_proj): Linear(in_features=16, out_features=16, bias=True)
        (out_proj): Linear(in_features=16, out_features=16, bias=True)
      )
      

In [26]:
# test bart_decoder
input_embeds = torch.randn(2, 4, config.d_model)
attention_mask = torch.randint(0, 2, (2, 4))
encoder_hidden_states = torch.randn(2, 4, config.d_model)
encoder_attention_mask = torch.randint(0, 2, (2, 4))
output = bart_decoder(
    input_embeds=input_embeds,
    attention_mask=attention_mask,
    encoder_hidden_states=encoder_hidden_states,
    encoder_attention_mask=encoder_attention_mask,
)
print(output.shape)

torch.Size([2, 4, 16])


In [27]:
from bart_model_from_scratch.model_seq2seq import BartSeq2seq
import torch.nn as nn

In [28]:
model = BartSeq2seq(config)
model

BartSeq2seq(
  (inputs_embeds): BartEmbeds(
    (embed_tokens): Embedding(50265, 16, padding_idx=2)
    (embed_positions): Embedding(1024, 16)
  )
  (decoder_inputs_embeds): BartEmbeds(
    (embed_tokens): Embedding(50265, 16, padding_idx=2)
    (embed_positions): Embedding(1024, 16)
  )
  (encoder): BartEncoder(
    (dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=16, out_features=16, bias=True)
          (v_proj): Linear(in_features=16, out_features=16, bias=True)
          (q_proj): Linear(in_features=16, out_features=16, bias=True)
          (out_proj): Linear(in_features=16, out_features=16, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (activation_fn): GELU(approximate='none')
        (activation_dropout): Dropout(p=0.0, inplace=False)
 

In [29]:
# test model
input_ids = torch.randint(0, 10, (2, 4))
attention_mask = (input_ids != config.pad_token_id).to(torch.int64)
decoder_input_ids = torch.randint(0, 10, (2, 4))
decoder_attention_mask = (decoder_input_ids != config.pad_token_id).to(torch.int64)
print(input_ids.shape)
print(attention_mask.shape)
print(decoder_input_ids.shape)
print(decoder_attention_mask.shape)

torch.Size([2, 4])
torch.Size([2, 4])
torch.Size([2, 4])
torch.Size([2, 4])


In [30]:
out = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    decoder_input_ids=decoder_input_ids,
    decoder_attention_mask=decoder_attention_mask,
)
print(out)

tensor([[[-0.1565,  0.0515,  0.0264,  ..., -0.0720, -0.0294,  0.1270],
         [-0.1412,  0.0390,  0.0277,  ..., -0.0550, -0.0214,  0.1114],
         [-0.1214,  0.0116,  0.0260,  ..., -0.0756, -0.0678,  0.0531],
         [-0.0986, -0.0128,  0.0336,  ..., -0.1044, -0.0647,  0.1063]],

        [[ 0.0861, -0.0086, -0.0126,  ...,  0.0174, -0.0422, -0.0960],
         [ 0.0518,  0.0299, -0.0117,  ...,  0.0144, -0.0465, -0.0912],
         [ 0.0906, -0.0103, -0.0112,  ...,  0.0239, -0.0375, -0.0987],
         [ 0.0595, -0.0018, -0.0103,  ..., -0.0074, -0.0586, -0.0788]]],
       grad_fn=<ViewBackward0>)


In [31]:
encoder_out = model.get_encoder_out(
    input_ids=input_ids,
    attention_mask=attention_mask,
)
print(encoder_out.last_hidden_state)

tensor([[[-1.0280e+00,  8.8693e-01,  2.3429e-01,  1.8027e-01, -1.0213e-01,
           4.2868e-01, -4.7562e-01,  9.8743e-01,  1.4823e+00, -3.0026e-01,
          -4.1284e-01, -8.2776e-01,  1.8947e+00, -3.1237e-01, -2.2919e-01,
          -2.4064e+00],
         [ 2.8718e-01,  9.6391e-01,  7.0305e-01, -2.7565e-01, -1.4134e-01,
           4.8724e-01, -5.8417e-01,  1.1320e+00,  1.7588e+00, -3.6381e-01,
          -4.6213e-01, -1.0921e+00,  3.2876e-01,  2.5883e-01, -2.2959e-01,
          -2.7710e+00],
         [-1.0619e+00,  1.2940e+00,  9.2832e-01, -7.7382e-02,  8.9972e-02,
           7.2326e-01, -4.4333e-01,  1.4173e+00,  4.2797e-01, -1.7117e-01,
          -3.1551e-01, -9.3816e-01,  5.5609e-01,  4.7057e-01, -9.6243e-02,
          -2.8038e+00],
         [-9.6978e-01,  8.6515e-01,  5.8152e-01, -2.6557e-01,  1.4826e-01,
           4.2492e-01, -5.5264e-01,  9.7091e-01,  1.4558e+00, -3.1750e-01,
          -4.0760e-01, -8.4619e-01,  1.8706e+00, -3.4542e-01, -2.2946e-01,
          -2.3829e+00]],

  

In [32]:
decoder_out = model.get_decoder_out(
    input_ids=decoder_input_ids,
    attention_mask=decoder_attention_mask,
    encoder_hidden_states=encoder_out.last_hidden_state,
    encoder_attention_mask=attention_mask
)
print(decoder_out.last_hidden_state)

tensor([[[ 0.5736, -1.4813,  1.3059,  2.0250, -0.9025, -1.4694, -0.7691,
          -0.9636, -0.2674,  1.2345,  0.3042, -0.5916,  0.0626,  0.0927,
          -0.1964,  1.0428],
         [-0.2215, -0.2653,  1.4288,  2.3012, -1.3321, -0.2015, -1.1367,
          -1.3015, -0.4530,  1.2909,  0.2138, -0.8577, -0.0424, -0.0749,
          -0.3334,  0.9853],
         [ 0.7770, -1.6532,  1.6490,  0.0758, -0.9033, -1.5593, -0.7597,
          -0.9696, -0.1642,  1.6285,  0.4898, -0.5231,  0.2482,  0.2717,
           0.0343,  1.3580],
         [ 0.6811, -1.5119,  1.4260,  2.1407, -0.8595, -1.4520, -0.7183,
          -0.9457, -0.2241,  1.3537,  0.3898, -0.5356,  0.1296,  0.1727,
          -0.0851,  0.0386]],

        [[ 0.2736,  0.1025, -0.1866, -0.0939,  1.6130, -1.3404, -0.5224,
          -0.5799, -0.2519, -1.0836,  0.1243,  0.7459,  1.8023, -0.0640,
           1.4122, -1.9511],
         [ 0.5043,  0.0218, -0.8330, -1.3302,  1.4892, -0.9727, -0.3571,
          -0.3810, -0.0162, -0.7692,  0.1619,  0.7